In [52]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
import requests

!pip install lxml
from lxml import html


     |████████████████████████████████| 5.5MB 17.3MB/s eta 0:00:01     |███▊                            | 634kB 17.3MB/s eta 0:00:01


In [53]:

# Toronto Open Data Catalogue - Neighbourhood Profiles 2016 (CSV)
# https://www.toronto.ca/city-government/data-research-maps/open-data/open-data-catalogue/#8c732154-5012-9afe-d0cd-ba3ffc813d5a

Tordf = pd.read_csv('https://www.toronto.ca/ext/open_data/catalog/data_set_files/2016_neighbourhood_profiles.csv', encoding='latin1')
latlons = pd.read_csv('http://cocl.us/Geospatial_data')


In [88]:
#scraping for postal codes and cleaning it and then making it into a dataframe
page=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
tree = html.fromstring(page.content)
postal_code = tree.xpath('//*[@id="mw-content-text"]/div/table[1]/tbody/tr/td[1]/text()')
borough = tree.xpath('//*[@id="mw-content-text"]/div/table[1]/tbody/tr/td[2]/text()')
neighborhood = tree.xpath('//*[@id="mw-content-text"]/div/table[1]/tbody/tr/td[3]/text()')
for i in range(len(postal_code)):
    postal_code[i] = postal_code[i].strip()
    borough[i] = borough[i].strip()
    neighborhood[i] = neighborhood[i].strip()
    if borough[i] == 'Not assigned':
        borough[i] = np.nan
    if neighborhood[i] == 'Not assigned':
        neighborhood[i] = borough[i]
df = pd.DataFrame({'postal_code': postal_code, 'Borough': borough, 'Neighborhood':neighborhood})
df.dropna(inplace=True)
latlons = pd.read_csv('http://cocl.us/Geospatial_data')
#merging two data frames to get latitude and longitude of each neighborhood
new_df = pd.merge(df, latlons, how='left', left_on = 'postal_code', right_on = 'Postal Code')
new_df = new_df.drop(columns = ['Postal Code', 'Borough', 'postal_code'])


In [117]:
#Separating each neighborhood 
newrows=[]
for i in range(len(new_df)):
    x=new_df.loc[i]
    y=new_df.loc[i][0].split(',')
    if len(y)> 1:
        for i in range(len(y)):
            new = [y[i], x[1], x[2]]
            newrows.append(new)
    else:
        newrows.append([x[0], x[1], x[2]])          
newer_df= pd.DataFrame(newrows, columns=['Neighborhood', 'Latitude', 'Longitude'])
newer_df

,Neighborhood,Latitude,Longitude
0,Parkwoods,43.753259,-79.329656
1,Victoria Village,43.725882,-79.315572
2,Regent Park,43.654260,-79.360636
3,Harbourfront,43.654260,-79.360636
4,Lawrence Manor,43.718518,-79.464763
5,Lawrence Heights,43.718518,-79.464763
6,Queen's Park,43.662301,-79.389494
7,Ontario Provincial Government,43.662301,-79.389494
8,Islington Avenue,43.667856,-79.532242
9,Humber Valley Village,43.667856,-79.532242


In [123]:
#merging the dataframes so that I can get the latitude and longitudes with the population and average incomes
Neighbourhoods = list(Tordf.columns.values)
Neighbourhoods = Neighbourhoods[5:]
pops = []
incomes = []
for i in range(len(Neighbourhoods)):
    pops.append(Tordf.iloc[2][i+4])
    incomes.append(Tordf.iloc[2266][i+4])
Tor_df = pd.DataFrame({'Neighborhood': Neighbourhoods, 'Population':pops, 'Average Income':incomes})
Tor_df.head()
newdf = pd.merge(Tor_df, newer_df, how='right', left_on = 'Neighborhood', right_on = 'Neighborhood')
newdf.head()

,Neighborhood,Population,Average Income,Latitude,Longitude
0,Alderwood,"23,757","16,490",43.602414,-79.543484
1,Bathurst Manor,"27,695","20,470",43.754328,-79.442259
2,Bayview Village,"25,797","19,170",43.786947,-79.385975
3,Dorset Park,"27,051","18,335",43.757410,-79.273304
4,Guildwood,"14,417","9,685",43.763573,-79.188711


In [125]:
#map the ones with the most extreme population and income
map_all = folium.Map(location=[43.6532, -79.3832], zoom_start=11)
color = ['red', 'blue', 'green', 'purple', 'grey', 'black', 'orange', 'yellow', 'white', 'brown']
for j in range(10):
    x = newdf.loc[j]
    label = 'Population: '+ str(x[1]) + 'Income: ' + str(x[2])
    lat = x[3]
    lng=x[4]
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker([lat, lng],radius=5,popup=label,color=color[j],fill=True,fill_color='#3186cc',fill_opacity=0.7,parse_html=False).add_to(map_all)
map_all